In [24]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

llm = ChatOpenAI(temperature = 0.1,
                 streaming= True,
                 callbacks=[StreamingStdOutCallbackHandler()]
                 )

examples = [
    {
        "movie_title": "Top Gun",
        "answer": "🛩️👨‍✈️🔥"
    },{
        "movie_title": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝"
    },{
        "movie_title": "Titanic",
        "answer": "🚢💑🧊"
    }
]

memory = ConversationSummaryBufferMemory(
    llm= llm, # 사용되는 LLM
    max_token_limit= 120, # 요약전 최대길이, 120토큰 초과시 요약한 정보
    return_messages=True, #실제 메세지로 바꿔달라, 그냥 텍스트가 아닌
)

example_prompt = ChatPromptTemplate.from_messages([
    ("human","Express this movie {movie_title} using emojis."),
    ("ai","{answer}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system","You are an artist, answer Just using 3 emojis"),
    MessagesPlaceholder(variable_name="history"),
    few_shot_prompt,
    ("human","Express this movie {movie_title} using emojis.")
])

def load_memory(t):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(movie_title):
    result = chain.invoke(
        {
            "movie_title":movie_title
        }
    )
    memory.save_context({"input":movie_title},{"output":result.content})
    print(result)


In [26]:
invoke_chain("Avater")

🌎🌿💙content='🌎🌿💙'


In [27]:
invoke_chain("나홀로집에")

🏠👦🏼🎄content='🏠👦🏼🎄'


In [30]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Avater'),
  AIMessage(content='🌎🌿💙'),
  HumanMessage(content='나홀로집에'),
  AIMessage(content='🏠👦🏼🎄'),
  HumanMessage(content='Avater'),
  AIMessage(content='🌿🌌👽')]}

In [29]:
invoke_chain("Avater")

🌿🌌👽content='🌿🌌👽'


In [31]:
question_prompt = ChatPromptTemplate.from_messages([
    ("system","You are an artist, answer Just using 3 emojis"),
    MessagesPlaceholder(variable_name="history"),
    ("human","What are {movie_title} emojis?")
])

question_chain = RunnablePassthrough.assign(history=load_memory) | question_prompt | llm

chain.invoke({"movie_title":"나홀로집에"})

🏠👦🏼🎄

AIMessageChunk(content='🏠👦🏼🎄')